In [1]:
!pip install langchain
!pip install langchain-community
!pip install tiktoken
!pip install chromadb
!pip install python-dotenv
!pip install langchain-google-genai
!pip install pdfminer.six

ERROR: Could not find a version that satisfies the requirement langchai-community (from versions: none)
ERROR: No matching distribution found for langchai-community
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 78.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 75.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 88.0 MB/s et

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 53.6 MB/s eta 0:00:00
ERROR: Operation cancelled by user


In [32]:
from dotenv import load_dotenv
load_dotenv()
import os
google_api_key = os.getenv("GOOGLE_API_KEY")
access_token = os.getenv("GOOGLE_API_KEY")
# Set the environment variable
os.environ["GOOGLE_API_KEY"] = google_api_key
os.environ["ACCESS_TOKEN"] = access_token

## Load PDF

In [2]:
from langchain_community.document_loaders import PDFMinerLoader

In [3]:
def load_pdf(path):
  pdf_loader = PDFMinerLoader(path)
  docs = pdf_loader.load()
  return docs

## Split PDF into Chunks

In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [5]:
def split_text(docs, chunk_size=2000, chunk_overlap=100):
  text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
  chunks = text_splitter.split_documents(docs)
  return chunks

## Generate Embedings and Store into ChormaDB

In [6]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.vectorstores import Chroma

In [13]:
def embd_and_store(chunks):
  embd = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
  vector_store = Chroma.from_documents(chunks,embd,persist_directory="./chroma")
  return vector_store

In [14]:
docs = load_pdf("./tesi.pdf") # Add path to your pdf
chunks = split_text(docs)
vector_store = embd_and_store(chunks)

## Get Embeddings


In [17]:
def get_embd(query):
  embd = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
  return embd.embed_query(query)

In [29]:
''' query = "What does the document say about AI?"
retrieved_docs = vector_store.similarity_search_with_score(query=query, k=5)

# Display results with distances
for i, (doc, score) in enumerate(retrieved_docs, 1):
    print(f"Result {i}:")
    print(f"Text: {doc.page_content}")
    print(f"Distance Score: {score}\n")
'''

'query = "What does the document say about AI?"\nretrieved_docs = vector_store.similarity_search_with_score(query=query, k=5)\n\n# Display results with distances\nfor i, (doc, score) in enumerate(retrieved_docs, 1):\n    print(f"Result {i}:")\n    # print(f"Text: {doc.page_content}")\n    print(f"Distance Score: {score}\n")\n'

## LLM Model

In [30]:
from transformers import AutoModelForCausalLM,AutoTokenizer

In [31]:
model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"

In [34]:
model = AutoModelForCausalLM.from_pretrained(model_name,trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained(model_name,trust_remote_code=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/679 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.55G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/3.07k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

In [62]:
def get_response(query):
    retrived_doc = vector_store.similarity_search(query=query,k=1)
    context = ""
    for doc in retrived_doc:
        context += doc.page_content + "\n"
    prompt = f"""
[System]: You are a helpful AI assistant. Provide concise and accurate answers to user questions.based on the context you recoved form the document provided.
[Context]: {context}
[User]: {query}
"""
    inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True)

    output = model.generate(
    inputs.input_ids,
    attention_mask=inputs.attention_mask,  # Pass the attention mask
    pad_token_id=tokenizer.pad_token_id,
    # max_length=100,  # Limit the response length
    temperature=0.7,  # Balances creativity and determinism
    top_p=0.9,       # Nucleus sampling: keeps the top 90% of probability mass
    top_k=50,        # Limits sampling to the top 50 tokens
    num_beams=1,     # Beam search for better coherence
    do_sample=True,  # Enables sampling for more diverse outputs
    repetition_penalty=1.2,  # Reduces repetition in the output
    no_repeat_ngram_size=2,  # Prevents repeating 2-grams
    early_stopping=True,     # Stops generation when the model is confident
)
    response = tokenizer.decode(output[0], skip_special_tokens=True)
    return response

In [65]:
query = "what is similarity metrics"
response = get_response(query)

In [66]:
print(response)


[System]: You are a helpful AI assistant. Provide concise and accurate answers to user questions.based on the context you recoved form the document provided.
[Context]: using the formula:

Cosine Similarity =

A · B
∥A∥∥B∥

where A and B are the vectors being compared, A · B is the dot product, and
∥A∥ and ∥B∥ are their norms. This metric is useful for assessing how aligned
two vectors are in the same direction, regardless of their magnitude, and is
often used to measure semantic similarity between words or documents.

9
Related Work

• Euclidean distance measures the linear distance between two vectors in

multidimensional space and is calculated using the formula:

Euclidean Distance =

ö
õ
õ
ô

n
Ø

i=1

(ai − bi)2

where ai and bi are the component values of vectors A and B, and n is
the number of dimensions in the vector. This metric provides an absolute
measure of the distance between the two vectors, useful for identifying overall
differences in their representations.

• maxim